In [3]:
# -*- coding: utf-8 -*-
import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import scipy.integrate as integrate
from __future__ import unicode_literals
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.mixture import GaussianMixture as gm
from multiprocessing import Pool
from sklearn.cluster import KMeans

In [170]:
def testujDecisionFunction(xTest, yTest, najDecFun, clf):
    wynik = clf.decision_function(xTest).reshape(yTest.shape)
    yTestWiekszeOdZera = yTest > 0
    if (najDecFun[1]):
        return np.sum((wynik > najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]
    return np.sum((wynik < najDecFun[0]) == yTestWiekszeOdZera) / yTest.shape[0]

def najlepszeDecFun(yTrain, wynikDecFun):
    yTrainWiekszeOdZera = yTrain > 0
    tabWynikow = []
    for i in range(wynikDecFun.shape[0]):
        wy = np.sum((wynikDecFun < wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
        wy = np.sum((wynikDecFun > wynikDecFun[i]) == yTrainWiekszeOdZera) / yTrain.shape[0]
        tabWynikow.append(wy)
    pozycjaMaxArg = np.argmax(tabWynikow)
    if (pozycjaMaxArg % 2 == 0):
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 0)
    else:
        return (wynikDecFun[int(pozycjaMaxArg / 2)], 1)

def standaryzowanie(tab):
    tabS = prep.StandardScaler().fit_transform(tab)
    tabN = prep.Normalizer().fit_transform(tab)
    tabSN = prep.Normalizer().fit_transform(tabS)
    tabNS = prep.StandardScaler().fit_transform(tabN)
    return tab, tabS, tabN, tabSN, tabNS

def crossValidationPoKolei(clf, data, dataClass):
    err = 0
    for i in range(data.shape[0]):
        pojedynczyErr = 0
        ilosc = 0
        for j in range(data.shape[1]):
            X_train = data[i][j][0]
            y_train = dataClass[i][j][0]
            X_test = data[i][j][1]
            y_test = dataClass[i][j][1] 
            pojIlosc = len(y_test)
            ilosc += pojIlosc
            pojedynczyErr += pojIlosc * licz(X_train, X_test, y_train, y_test, clf)

        err += pojedynczyErr / ilosc
    return float(err) / data.shape[0]

def przelicz(xTrain, xTest, yTrain, yTest, kmeans, clf, kmeansData):
    trainLabels = kmeansData[0]
    testLabels = kmeansData[1]
    k = kmeans.n_clusters
    wynik = 0
    for i in range(k):
        iloscTestowych = np.sum(testLabels == i)
        if (np.sum(trainLabels == i) > 1 and iloscTestowych > 1):
            wynik += iloscTestowych * licz(xTrain[trainLabels==i], xTest[testLabels==i], yTrain[trainLabels==i], yTest[testLabels==i], clf)
    return wynik / yTest.shape[0]

def licz(xTrain, xTest, yTrain, yTest, clf):
    if(len(yTest) == 0):
        return 0
    if (len(yTrain) == 0):
        wynik = np.sum(yTest == 1) / yTest.shape[0]
        return wynik
    if (np.unique(yTrain).shape[0] == 1):
        wynik = np.sum(yTest == yTrain[0]) / yTest.shape[0]
        return wynik
    
    clf = clf.fit(xTrain, yTrain)
    yTrain = yTrain.reshape((yTrain.shape[0], 1))
    wynikDecFun = clf.decision_function(xTrain).reshape(yTrain.shape)
    najDecFun = najlepszeDecFun(yTrain, wynikDecFun)  
    return testujDecisionFunction(xTest, yTest, najDecFun, clf)

def generujPodzialy(tab, p = 0.75):
    podzial = []
    n = len(tab)
    k = int(10*n/100)
    dlugosc = int(4/10*n)
    dlugoscUczaca = int(4/10*p*n)
    for i in range(6):
        uczace = tab[np.arange(i*k, min(i*k + dlugoscUczaca, n))]
        testowe = tab[np.arange(min(i*k + dlugoscUczaca, n), min((i*k + dlugosc), n))]
        podzial.append([uczace, testowe])
    return podzial

def stworzGrupowanie(kmeans, tabDlaKmeans, tabDlaSvm, zmiana):
    podzialSvm = generujPodzialy(tabDlaSvm)
    podzialKmeans = generujPodzialy(tabDlaKmeans)
    podzialZmiana = generujPodzialy(zmiana)
    
    wynik = []
    wynikZmiana = []
    for j in range(len(podzialKmeans)):
        xKmeansTrain = podzialKmeans[j][0]
        xKmeansTest = podzialKmeans[j][1]
        zmianaTrain = podzialZmiana[j][0]
        zmianaTest = podzialZmiana[j][1]
        svmTrain = podzialSvm[j][0]
        svmTest = podzialSvm[j][1]
        kmeans.fit(xKmeansTrain)
        trainLabels = kmeans.labels_
        testLabels = kmeans.predict(xKmeansTest)
        pojedynczyWynik = []
        pojedynczyWynikZmiana = []
        for i in range(kmeans.n_clusters):
            pojedynczyWynik.append([svmTrain[i == trainLabels], svmTest[i == testLabels]])
            pojedynczyWynikZmiana.append([zmianaTrain[i == trainLabels], zmianaTest[i == testLabels]])
        wynik.append(pojedynczyWynik)
        wynikZmiana.append(pojedynczyWynikZmiana)
    return np.array(wynik), np.array(wynikZmiana)

def stworzGrupowanieLosowe(tabDlaSvm, zmiana, rand, k):
    podzialSvm = generujPodzialy(tabDlaSvm)
    podzialZmiana = generujPodzialy(zmiana)
    
    wynik = []
    wynikZmiana = []
    for j in range(len(podzialSvm)):
        zmianaTrain = podzialZmiana[j][0]
        zmianaTest = podzialZmiana[j][1]
        svmTrain = podzialSvm[j][0]
        svmTest = podzialSvm[j][1]
        trainLabels = rand.randint(0, k, len(zmianaTrain))
        testLabels = rand.randint(0, k, len(zmianaTest))
        pojedynczyWynik = []
        pojedynczyWynikZmiana = []
        for i in range(k):
            pojedynczyWynik.append([svmTrain[i == trainLabels], svmTest[i == testLabels]])
            pojedynczyWynikZmiana.append([zmianaTrain[i == trainLabels], zmianaTest[i == testLabels]])
        wynik.append(pojedynczyWynik)
        wynikZmiana.append(pojedynczyWynikZmiana)
        
    return np.array(wynik), np.array(wynikZmiana)

def rodzaj(x):
    return {
        0 : 'Czyste',
        1 : 'S',
        2 : 'N',
        3 : 'S->N',
        4 : 'N->S'
    }[x]

In [118]:
rand = np.random().randint
rand.set_state(3)
rand(0, 2, 3)

AttributeError: 'builtin_function_or_method' object has no attribute 'set_state'

In [169]:
rand = np.random.RandomState(1234)
print (rand.randint(0,3,5))
print (rand.randint(0,3,6))

[2 1 0 0 0]
[1 1 1 2 2 2]


In [168]:
rand = np.random.RandomState(1234)
for i in range(10):
    print (rand.randint(0,3,5))

[2 1 0 0 0]
[1 1 1 2 2]
[2 0 0 2 2]
[2 0 0 0 1]
[0 1 2 2 2]
[0 0 1 2 2]
[2 0 1 0 2]
[0 1 2 1 2]
[0 2 2 0 1]
[1 0 0 0 1]


In [176]:
def test(mozliwosc, svm):
    wynik1 = crossValidationPoKolei(svm, mozliwosc[0][0], mozliwosc[0][1])
    wynik2 = crossValidationPoKolei(svm, mozliwosc[1][0], mozliwosc[1][1])
    wynik3 = crossValidationPoKolei(svm, mozliwosc[2][0], mozliwosc[2][1])
    wynik4= crossValidationPoKolei(svm, mozliwosc[3][0], mozliwosc[3][1])
    wynik5 = crossValidationPoKolei(svm, mozliwosc[4][0], mozliwosc[4][1])
    crossValidationPoKolei(svm, gPodzial, gZmiana)
    return wynik1, wynik2, wynik3, wynik4, wynik5

def wylicz(tabSvm, tabKmeans, zmiana, tabParametrow, k, iloscTestow):
    listaTabKmeans = standaryzowanie(tabKmeans)
    listaTabSvm = standaryzowanie(tabSvm)
    
    tablicaTestow = []
    for a in range(iloscTestow):
        tablicaMozliwosci = []
        for i in range(5):
            mozliwosc = []
            for j in range(5):
                kmeans = KMeans(init='k-means++', n_clusters=k, n_init=15, random_state=a*12345)
                mozliwosc.append(stworzGrupowanie(kmeans, listaTabKmeans[i], listaTabSvm[j], zmiana))
            tablicaMozliwosci.append(mozliwosc)
        tablicaTestow.append(tablicaMozliwosci)
    
    tabWynikow = np.zeros((len(tabParametrow)*5,5), dtype=np.float32)

    for tablicaMozliwosci in tablicaTestow:
        i = 0
        for parametry in tabParametrow:
            gamma = parametry[0]
            C = parametry[1]
            j = 0
            for mozliwosc in tablicaMozliwosci:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C, random_state=1)
                wynikTestu = test(mozliwosc, svm)
                tabWynikow[i+j] += wynikTestu
                j += 1
            i  += 5
    return tabWynikow / iloscTestow

def wyliczLosowe(tabSvm, zmiana, tabParametrow, k, iloscTestow):
    listaTabSvm = standaryzowanie(tabSvm)
    rand = np.random.RandomState(1)
    tablicaTestow = []
    for a in range(iloscTestow):
        tablicaMozliwosci = []
        for i in range(5):
            mozliwosc = []
            for j in range(5):
                mozliwosc.append(stworzGrupowanieLosowe(listaTabSvm[j], zmiana, rand, k))
            tablicaMozliwosci.append(mozliwosc)
        tablicaTestow.append(tablicaMozliwosci)
    
    tabWynikow = np.zeros((len(tabParametrow)*5,5), dtype=np.float32)

    for tablicaMozliwosci in tablicaTestow:
        i = 0
        for parametry in tabParametrow:
            gamma = parametry[0]
            C = parametry[1]
            j = 0
            for mozliwosc in tablicaMozliwosci:
                svm = SVC(class_weight='balanced', gamma = gamma, kernel = 'rbf', C = C, random_state=1)
                wynikTestu = test(mozliwosc, svm)
                tabWynikow[i+j] += wynikTestu
                j += 1
            i  += 5
    return tabWynikow / iloscTestow

def wypisz(tabWynikow):
    print('Czyste     S      N     S->N    N->S')
    i = 0
    for parametry in tabParametrow:
        gamma = parametry[0]
        C = parametry[1]
        for j in range(5):
            wynik1, wynik2, wynik3, wynik4, wynik5 = tabWynikow[i+j]
            odpowiedz = "{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}".format(wynik1, wynik2, wynik3, wynik4, wynik5)
            if (wynik1 >= 0.544 or wynik2 >= 0.544 or wynik3 >= 0.544 or wynik4 > 0.544 or wynik5 >= 0.544):
                odpowiedz = "\x1b[31m\"{0:0.4f}  {1:0.4f}  {2:0.4f}  {3:0.4f}  {4:0.4f}\"\x1b[0m".format(wynik1, wynik2, wynik3, wynik4, wynik5)
            print(odpowiedz, "- dla: g:", gamma, "C:", C, "daneKmeans:", rodzaj(j)) 
        print()
        i += 5

In [67]:
gmm_10_M_fPierw = np.load("gmm_10_M_fPierw.npy")
zmiana = np.load("zmiana.npy")
imbalance = np.load("imbalance.npy")

gammas = [1e-6, 1e-3, 1, 1e3]
Cs = [1e-6, 1e-3, 1, 1e3]
tabParametrow = np.zeros((len(gammas)*len(Cs),2), dtype=np.float64)
i = 0
for gamma in gammas:
    for C in Cs:
        tabParametrow[i][0] = gamma
        tabParametrow[i][1] = C
        i += 1

In [179]:
t = time.time()
losowe2_imbalanceWynik = wyliczLosowe(imbalance, zmiana, tabParametrow, 2, 10)
print(time.time() - t)
np.save("losowe2_imbalanceWynik.npy", losowe2_imbalanceWynik)
wypisz(losowe2_imbalanceWynik)

1920.4363079071045
Czyste     S      N     S->N    N->S
0.5374  0.5421  0.5411  0.5397  0.5391 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5380  0.5391  0.5410  0.5391  0.5388 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5397  0.5407  0.5408  0.5397  0.5401 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5395  0.5401  0.5399  0.5390  0.5380 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5385  0.5407  0.5376  0.5374  0.5392 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5374  0.5421  0.5411  0.5397  0.5391 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5380  0.5391  0.5410  0.5391  0.5388 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5397  0.5407  0.5408  0.5397  0.5401 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5395  0.5401  0.5399  0.5390  0.5380 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5385  0.5407  0.5376  0.5374  0.5392 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5374  0.5421  0.5411  0.5397  0.5391 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5380  0.5391  0.5410  0.5391  0.5388 - dla: g: 1e-06 C: 1

In [194]:
t = time.time()
losowe4_imbalanceWynik = wyliczLosowe(imbalance, zmiana, tabParametrow, 2, 10)
print(time.time() - t)
np.save("losowe4_imbalanceWynik.npy", losowe4_imbalanceWynik)
wypisz(losowe4_imbalanceWynik)

1962.3922410011292
Czyste     S      N     S->N    N->S
0.5374  0.5421  0.5411  0.5397  0.5391 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5380  0.5391  0.5410  0.5391  0.5388 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5397  0.5407  0.5408  0.5397  0.5401 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5395  0.5401  0.5399  0.5390  0.5380 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5385  0.5407  0.5376  0.5374  0.5392 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5374  0.5421  0.5411  0.5397  0.5391 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5380  0.5391  0.5410  0.5391  0.5388 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5397  0.5407  0.5408  0.5397  0.5401 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5395  0.5401  0.5399  0.5390  0.5380 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5385  0.5407  0.5376  0.5374  0.5392 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5374  0.5421  0.5411  0.5397  0.5391 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5380  0.5391  0.5410  0.5391  0.5388 - dla: g: 1e-06 C: 1

In [180]:
t = time.time()
means2_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 2, 10)
print(time.time() - t)
np.save("means2_imbalance_gmm_10_test3.npy", means2_imbalance_gmm_10Wynik)
wypisz(means2_imbalance_gmm_10Wynik)

1931.4545757770538
Czyste     S      N     S->N    N->S
"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5276  0.5276  0.5287  0.5327  0.5287 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5410  0.5410  0.5359  0.5356  0.5359 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5280  0.5280  0.5330  0.5327  0.5330 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5276  0.5276  0.5287  0.5327  0.5287 - dla: g: 1e-06 C: 0.001 daneKmeans: S
"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5410  0.5410  0.5359  0.5356  0.5359 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5280  0.5280  0.5330  0.5327  0.5330 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

"0.5450  0.5450  0.5428  0.5421  0.5428" - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5276  0.5276  0.5287  0.5327  0.5287 - dla: g: 

In [181]:
t = time.time()
means3_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 3, 10)
print(time.time() - t)
np.save("means3_imbalance_gmm_10_test3.npy", means3_imbalance_gmm_10Wynik)
wypisz(means3_imbalance_gmm_10Wynik)

1428.1227657794952
Czyste     S      N     S->N    N->S
0.5352  0.5352  0.5418  0.5428  0.5418 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5310  0.5310  0.5374  0.5367  0.5374 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5348  0.5348  0.5410  0.5424  0.5410 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5347  0.5347  0.5200  0.5196  0.5200 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5305  0.5305  0.5378  0.5367  0.5378 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5352  0.5352  0.5418  0.5428  0.5418 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5310  0.5310  0.5374  0.5367  0.5374 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5348  0.5348  0.5410  0.5424  0.5410 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5347  0.5347  0.5200  0.5196  0.5200 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5305  0.5305  0.5378  0.5367  0.5378 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5352  0.5352  0.5418  0.5428  0.5418 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5310  0.5310  0.5374  0.5367  0.5374 - dla: g: 1e-06 C: 1

In [182]:
t = time.time()
means4_imbalance_gmm_10Wynik = wylicz(imbalance,(( gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 4, 10)
print(time.time() - t)
np.save("means4_imbalance_gmm_10_test3.npy", means4_imbalance_gmm_10Wynik)
wypisz(means4_imbalance_gmm_10Wynik)

1256.0500411987305
Czyste     S      N     S->N    N->S
"0.5443  0.5443  0.5374  0.5385  0.5374" - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5385  0.5385  0.5350  0.5354  0.5350 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5437  0.5437  0.5364  0.5375  0.5364 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5308  0.5308  0.5242  0.5235  0.5242 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5395  0.5395  0.5363  0.5360  0.5363 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

"0.5443  0.5443  0.5374  0.5385  0.5374" - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5385  0.5385  0.5350  0.5354  0.5350 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5437  0.5437  0.5364  0.5375  0.5364 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5308  0.5308  0.5242  0.5235  0.5242 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5395  0.5395  0.5363  0.5360  0.5363 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

"0.5443  0.5443  0.5374  0.5385  0.5374" - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5385  0.5385  0.5350  0.5354  0.5350 - dla: g: 1e-0

In [183]:
t = time.time()
means5_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 5 , 10)
print(time.time() - t)
np.save("means5_imbalance_gmm_10_test3.npy", means5_imbalance_gmm_10Wynik)
wypisz(means5_imbalance_gmm_10Wynik)

1149.0054905414581
Czyste     S      N     S->N    N->S
0.5259  0.5259  0.5362  0.5397  0.5362 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5393  0.5393  0.5304  0.5293  0.5304 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5260  0.5260  0.5378  0.5403  0.5378 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5293  0.5293  0.5262  0.5258  0.5262 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5387  0.5387  0.5286  0.5267  0.5286 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5259  0.5259  0.5362  0.5397  0.5362 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5393  0.5393  0.5304  0.5293  0.5304 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5260  0.5260  0.5378  0.5403  0.5378 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5293  0.5293  0.5262  0.5258  0.5262 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5387  0.5387  0.5286  0.5259  0.5286 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5259  0.5259  0.5362  0.5397  0.5362 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5393  0.5393  0.5304  0.5293  0.5304 - dla: g: 1e-06 C: 1

In [184]:
t = time.time()
means6_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 6, 10)
print(time.time() - t)
np.save("means6_imbalance_gmm_10_test3.npy", means6_imbalance_gmm_10Wynik)
wypisz(means6_imbalance_gmm_10Wynik)

1059.3962070941925
Czyste     S      N     S->N    N->S
0.5216  0.5216  0.5365  0.5358  0.5365 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5140  0.5140  0.5224  0.5227  0.5224 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5206  0.5206  0.5360  0.5309  0.5360 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5249  0.5249  0.5209  0.5201  0.5209 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5158  0.5158  0.5231  0.5248  0.5231 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5216  0.5216  0.5365  0.5358  0.5365 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5140  0.5140  0.5224  0.5227  0.5224 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5206  0.5206  0.5360  0.5309  0.5360 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5249  0.5249  0.5209  0.5201  0.5209 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5158  0.5158  0.5231  0.5248  0.5231 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5216  0.5216  0.5365  0.5358  0.5365 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5140  0.5140  0.5224  0.5227  0.5224 - dla: g: 1e-06 C: 1

In [185]:
t = time.time()
means7_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 7, 10)
print(time.time() - t)
np.save("means7_imbalance_gmm_10_test3.npy", means7_imbalance_gmm_10Wynik)
wypisz(means7_imbalance_gmm_10Wynik)

1064.4097754955292
Czyste     S      N     S->N    N->S
0.5151  0.5151  0.5278  0.5279  0.5278 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5117  0.5117  0.5220  0.5207  0.5220 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5133  0.5133  0.5321  0.5293  0.5321 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5206  0.5206  0.5188  0.5178  0.5188 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5233  0.5233  0.5291  0.5260  0.5291 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5151  0.5151  0.5278  0.5279  0.5278 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5117  0.5117  0.5220  0.5207  0.5220 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5133  0.5133  0.5321  0.5293  0.5321 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5206  0.5206  0.5188  0.5178  0.5188 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5233  0.5233  0.5291  0.5260  0.5291 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5151  0.5151  0.5278  0.5279  0.5278 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5117  0.5117  0.5220  0.5207  0.5220 - dla: g: 1e-06 C: 1

In [186]:
t = time.time()
means8_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 8, 10)
print(time.time() - t)
np.save("means8_imbalance_gmm_10_test3.npy", means8_imbalance_gmm_10Wynik)
wypisz(means8_imbalance_gmm_10Wynik)

1043.177503824234
Czyste     S      N     S->N    N->S
0.5063  0.5063  0.5319  0.5327  0.5319 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5138  0.5138  0.5249  0.5235  0.5249 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5052  0.5052  0.5293  0.5294  0.5293 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5184  0.5184  0.5174  0.5166  0.5174 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5101  0.5101  0.5260  0.5249  0.5260 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5063  0.5063  0.5319  0.5327  0.5319 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5138  0.5138  0.5249  0.5235  0.5249 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5052  0.5052  0.5293  0.5294  0.5289 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5184  0.5184  0.5174  0.5166  0.5174 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5101  0.5101  0.5260  0.5249  0.5260 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5063  0.5063  0.5319  0.5327  0.5319 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5138  0.5138  0.5249  0.5235  0.5249 - dla: g: 1e-06 C: 1.

In [187]:
t = time.time()
means9_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 9, 10)
print(time.time() - t)
np.save("means9_imbalance_gmm_10_test3.npy", means9_imbalance_gmm_10Wynik)
wypisz(means9_imbalance_gmm_10Wynik)

1008.3593726158142
Czyste     S      N     S->N    N->S
0.5110  0.5110  0.5309  0.5326  0.5309 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5157  0.5157  0.5273  0.5251  0.5273 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5079  0.5079  0.5301  0.5315  0.5301 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5194  0.5194  0.5171  0.5162  0.5171 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5128  0.5128  0.5268  0.5245  0.5268 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5110  0.5110  0.5309  0.5326  0.5306 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5157  0.5157  0.5273  0.5251  0.5273 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5079  0.5079  0.5301  0.5315  0.5300 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5194  0.5194  0.5171  0.5162  0.5171 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5128  0.5128  0.5268  0.5245  0.5268 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5110  0.5110  0.5306  0.5329  0.5306 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5157  0.5157  0.5273  0.5251  0.5273 - dla: g: 1e-06 C: 1

In [188]:
t = time.time()
means10_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 10, 10)
print(time.time() - t)
np.save("means10_imbalance_gmm_10_test3.npy", means10_imbalance_gmm_10Wynik)
wypisz(means10_imbalance_gmm_10Wynik)

994.9984474182129
Czyste     S      N     S->N    N->S
0.5063  0.5063  0.5315  0.5338  0.5315 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5155  0.5155  0.5264  0.5245  0.5264 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5075  0.5075  0.5298  0.5330  0.5298 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5188  0.5188  0.5220  0.5202  0.5220 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5140  0.5140  0.5283  0.5241  0.5283 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5063  0.5063  0.5315  0.5338  0.5312 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5155  0.5155  0.5264  0.5245  0.5264 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5075  0.5075  0.5298  0.5330  0.5297 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5188  0.5188  0.5220  0.5202  0.5220 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5140  0.5140  0.5283  0.5241  0.5283 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5063  0.5063  0.5312  0.5338  0.5312 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5155  0.5155  0.5264  0.5245  0.5264 - dla: g: 1e-06 C: 1.

In [189]:
t = time.time()
means11_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 11, 10)
print(time.time() - t)
np.save("means11_imbalance_gmm_10_test3.npy", means11_imbalance_gmm_10Wynik)
wypisz(means11_imbalance_gmm_10Wynik)

983.1050009727478
Czyste     S      N     S->N    N->S
0.5114  0.5114  0.5279  0.5284  0.5279 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5183  0.5183  0.5301  0.5250  0.5301 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5104  0.5104  0.5297  0.5310  0.5297 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5158  0.5158  0.5227  0.5219  0.5227 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5170  0.5170  0.5293  0.5242  0.5293 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5115  0.5115  0.5279  0.5284  0.5279 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5183  0.5183  0.5301  0.5250  0.5301 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5104  0.5104  0.5297  0.5310  0.5297 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5158  0.5158  0.5227  0.5219  0.5227 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5170  0.5170  0.5293  0.5242  0.5293 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5115  0.5115  0.5279  0.5284  0.5279 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5183  0.5183  0.5299  0.5246  0.5299 - dla: g: 1e-06 C: 1.

In [190]:
t = time.time()
means12_imbalance_gmm_10Wynik = wylicz(imbalance, ((gmm_10_M_fPierw).T[:60]).T, zmiana, tabParametrow, 12, 10)
print(time.time() - t)
np.save("means12_imbalance_gmm_10_test3.npy", means12_imbalance_gmm_10Wynik)
wypisz(means12_imbalance_gmm_10Wynik)

955.236615896225
Czyste     S      N     S->N    N->S
0.5128  0.5128  0.5259  0.5235  0.5259 - dla: g: 1e-06 C: 1e-06 daneKmeans: Czyste
0.5172  0.5172  0.5323  0.5274  0.5323 - dla: g: 1e-06 C: 1e-06 daneKmeans: S
0.5136  0.5137  0.5237  0.5217  0.5237 - dla: g: 1e-06 C: 1e-06 daneKmeans: N
0.5141  0.5141  0.5184  0.5181  0.5184 - dla: g: 1e-06 C: 1e-06 daneKmeans: S->N
0.5215  0.5215  0.5298  0.5257  0.5298 - dla: g: 1e-06 C: 1e-06 daneKmeans: N->S

0.5128  0.5128  0.5260  0.5235  0.5260 - dla: g: 1e-06 C: 0.001 daneKmeans: Czyste
0.5171  0.5171  0.5324  0.5274  0.5324 - dla: g: 1e-06 C: 0.001 daneKmeans: S
0.5137  0.5137  0.5237  0.5217  0.5236 - dla: g: 1e-06 C: 0.001 daneKmeans: N
0.5139  0.5139  0.5184  0.5181  0.5184 - dla: g: 1e-06 C: 0.001 daneKmeans: S->N
0.5215  0.5215  0.5298  0.5257  0.5298 - dla: g: 1e-06 C: 0.001 daneKmeans: N->S

0.5128  0.5128  0.5260  0.5235  0.5259 - dla: g: 1e-06 C: 1.0 daneKmeans: Czyste
0.5171  0.5171  0.5323  0.5274  0.5323 - dla: g: 1e-06 C: 1.0